In [8]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score, recall_score
from scipy.sparse import csr_matrix
from math import sqrt
import ast

# Load and prepare the ratings data
with open('ratings.py', 'r') as file:
    ratings_content = file.read()
ratings_list = ast.literal_eval(ratings_content.split('=')[1].strip())
ratings_df = pd.DataFrame(ratings_list)

# Prepare the user-item matrix
pivot_table = ratings_df.pivot_table(index='user_id', columns='product_id', values='rating').fillna(0)
X = csr_matrix(pivot_table.values)

# Apply SVD
svd = TruncatedSVD(n_components=20, random_state=42)
X_reduced = svd.fit_transform(X)

# Inverse transform to get rating predictions
X_pred = svd.inverse_transform(X_reduced)

# Flatten the matrices for calculating regression metrics
original_ratings = X.toarray().flatten()
predicted_ratings = X_pred.flatten()

# Calculate RMSE, MSE, and MAE
mse = mean_squared_error(original_ratings, predicted_ratings)
rmse = sqrt(mse)
mae = mean_absolute_error(original_ratings, predicted_ratings)

# Define a threshold for classification
threshold = 3
actual_classes = [1 if rating > threshold else 0 for rating in original_ratings]
predicted_classes = [1 if rating > threshold else 0 for rating in predicted_ratings]

# Calculate accuracy, F1 score, and recall
accuracy = accuracy_score(actual_classes, predicted_classes)
f1 = f1_score(actual_classes, predicted_classes)
recall = recall_score(actual_classes, predicted_classes)

# Print evaluation metrics
print(f'RMSE: {rmse}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Recall: {recall}')


RMSE: 0.16819711862775977
MSE: 0.028290270714680696
MAE: 0.019132663565655243
Accuracy: 0.9984962406015038
F1 Score: 0.9727272727272727
Recall: 0.9553571428571429
